# *Ejercicio 2*

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split

# Cargar corpus
with open("corpus_features_balanced.json", "r", encoding="utf-8") as f:
    data = json.load(f)

texts = [
    (review.get("text_clean") or review.get("text_raw") or "").strip()
    for review in data
]

# Cargar etiquetas originales de práctica 2
y = np.load("y_total.npy", allow_pickle=True)

label2id = {"negative": 0, "neutral": 1, "positive": 2}
y = np.array([label2id[label] for label in y], dtype=np.int64)

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    texts, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader

MODEL_NAME = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
MAX_LEN = 128   # Mucho mejor que 20

class ReviewDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target = self.targets[idx]

        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "targets": torch.tensor(target, dtype=torch.long),
        }

train_dataset = ReviewDataset(X_train, y_train, tokenizer, MAX_LEN)
test_dataset  = ReviewDataset(X_test,  y_test,  tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=16)


In [ ]:
from transformers import BertModel
import torch.nn as nn

class SentimentClassifier(nn.Module):
    def __init__(self, n_classes=3):
        super().__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.pooler_output
        output = self.drop(pooled_output)
        return self.out(output)


In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SentimentClassifier().to(device)

EPOCHS = 3
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss().to(device)

total_steps = len(train_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler):
    model.train()
    losses = []
    correct = 0

    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        targets = batch["targets"].to(device)

        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, targets)

        _, preds = torch.max(outputs, dim=1)
        correct += torch.sum(preds == targets)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct.double() / len(data_loader.dataset), np.mean(losses)


def eval_model(model, data_loader, loss_fn, device):
    model.eval()
    losses = []
    correct = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            targets = batch["targets"].to(device)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, targets)

            _, preds = torch.max(outputs, dim=1)
            correct += torch.sum(preds == targets)
            losses.append(loss.item())

    return correct.double() / len(data_loader.dataset), np.mean(losses)


In [ ]:
best_acc = 0

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")

    train_acc, train_loss = train_epoch(
        model, train_loader, loss_fn, optimizer, device, scheduler
    )

    val_acc, val_loss = eval_model(
        model, test_loader, loss_fn, device
    )

    print(f"Train acc: {train_acc:.4f}, loss: {train_loss:.4f}")
    print(f"Val   acc: {val_acc:.4f}, loss: {val_loss:.4f}")

    if val_acc > best_acc:
        torch.save(model.state_dict(), "best_bert_model.bin")
        best_acc = val_acc


Epoch 1/3
Train acc: 0.5553, loss: 0.8959
Val   acc: 0.6753, loss: 0.7016
Epoch 2/3
Train acc: 0.7496, loss: 0.5950
Val   acc: 0.6999, loss: 0.6642
Epoch 3/3
Train acc: 0.8409, loss: 0.4185
Val   acc: 0.7122, loss: 0.6773


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
import torch

# Cargar modelo entrenado
model = SentimentClassifier().to(device)
model.load_state_dict(torch.load("best_bert_model.bin", map_location=device))
model.eval()

all_preds = []
all_targets = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        targets = batch["targets"].to(device)

        outputs = model(input_ids, attention_mask)
        _, preds = torch.max(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

# Métricas
print("Accuracy:", accuracy_score(all_targets, all_preds))
print("F1 macro:", f1_score(all_targets, all_preds, average="macro"))
print("F1 weighted:", f1_score(all_targets, all_preds, average="weighted"))

print("\nClassification report:")
print(classification_report(all_targets, all_preds, target_names=["negative","neutral","positive"]))

print("\nConfusion matrix:")
print(confusion_matrix(all_targets, all_preds))


Accuracy: 0.7122060470324748
F1 macro: 0.7125321994207185
F1 weighted: 0.7124840209703995

Classification report:
              precision    recall  f1-score   support

    negative       0.77      0.74      0.76       297
     neutral       0.60      0.61      0.61       298
    positive       0.77      0.78      0.78       298

    accuracy                           0.71       893
   macro avg       0.71      0.71      0.71       893
weighted avg       0.71      0.71      0.71       893


Confusion matrix:
[[221  63  13]
 [ 59 182  57]
 [  8  57 233]]
